In [ ]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, sum, avg
os.chdir('/Users/chkapsalis/Documents/GitHub/Big_Data_Architectures/Assignments/my_assignment_3')

# For some reason i need to run this every time in order to get it work
import os
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@11/libexec/openjdk.jdk/Contents/Home" 

# the .config("option", "value") arguments allow us to perform refined file I/O
spark = SparkSession.builder.master("local[1]") \
            .appName("app") \
            .config("option", "value") \
            .getOrCreate()

In [22]:
attacks = spark.read.csv('file:///' + os.getcwd() + '/Global_Cybersecurity_Threats_2015-2024.csv', header=True)
attacks.printSchema()  # everything is a string!!!
attacks.show()

root
 |-- Country: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Attack Type: string (nullable = true)
 |-- Target Industry: string (nullable = true)
 |-- Financial Loss (in Million $): string (nullable = true)
 |-- Number of Affected Users: string (nullable = true)
 |-- Attack Source: string (nullable = true)
 |-- Security Vulnerability Type: string (nullable = true)
 |-- Defense Mechanism Used: string (nullable = true)
 |-- Incident Resolution Time (in Hours): string (nullable = true)

+---------+----+-----------------+------------------+-----------------------------+------------------------+-------------+---------------------------+----------------------+-----------------------------------+
|  Country|Year|      Attack Type|   Target Industry|Financial Loss (in Million $)|Number of Affected Users|Attack Source|Security Vulnerability Type|Defense Mechanism Used|Incident Resolution Time (in Hours)|
+---------+----+-----------------+------------------+--------------

In [23]:
# I will now assign every column its proper data type and cache the resulting rdd to make sure 
# that spark does not implement the same transformations over and over again.
attacks = attacks \
    .select(
        attacks["Country"].alias("country"),
        attacks["Year"].cast("int").alias("year"),
        attacks["Attack Type"].alias("attack_type"),
        attacks["Financial Loss (in Million $)"].cast("float").alias("fin_loss"),
        attacks["Number of Affected Users"].cast("int").alias("no_affected_users"),
        attacks["Attack Source"].alias("attack_source"),
        attacks["Incident Resolution Time (in Hours)"].cast("int").alias("resolution_time"),
        attacks["Target Industry"].alias("target_industry")
    ).cache()

# attacks.withColumn("year", attacks["Year"].cast("int")) \
#     .withColumn("fin_loss", attacks["Financial Loss (in Million $)"].cast("float")) \
#     .withColumn("no_affected_users", attacks["Number of Affected Users"].cast("int")) \
#     .withColumn("resolution_time", attacks["Incident Resolution Time (in Hours)"].cast("int")) \
#     .withColumn("target_industry", attacks["Target Industry"])

In [24]:
attacks.printSchema()
attacks.show()

root
 |-- country: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- attack_type: string (nullable = true)
 |-- fin_loss: float (nullable = true)
 |-- no_affected_users: integer (nullable = true)
 |-- attack_source: string (nullable = true)
 |-- resolution_time: integer (nullable = true)
 |-- target_industry: string (nullable = true)

+---------+----+-----------------+--------+-----------------+-------------+---------------+------------------+
|  country|year|      attack_type|fin_loss|no_affected_users|attack_source|resolution_time|   target_industry|
+---------+----+-----------------+--------+-----------------+-------------+---------------+------------------+
|    China|2019|         Phishing|   80.53|           773169| Hacker Group|             63|         Education|
|    China|2019|       Ransomware|   62.19|           295961| Hacker Group|             71|            Retail|
|    India|2017|Man-in-the-Middle|   38.65|           605895| Hacker Group|             20|

In [ ]:
### Question 1: Calculate total financial loss per country
total_fin_loss_per_country = attacks \
    .select(
        col("country"),
        col("fin_loss")
    ) \
    .groupBy(col("country")) \
    .agg(sum("fin_loss").alias("total_fin_loss"))

print("Question 1: Total Financial Loss Per Country")
total_fin_loss_per_country.show()


# attacks.select("fin_loss").distinct().show(truncate=False)


Question 1: Total Financial Loss Per Country
+---------+------------------+
|  country|    total_fin_loss|
+---------+------------------+
|   Russia|14734.729979157448|
|  Germany|15793.239986002445|
|   France|14972.279991805553|
|    China| 13714.46997320652|
|    India|14566.120049655437|
|      USA|14812.119963765144|
|       UK| 16502.98998105526|
|   Brazil|15782.620014667511|
|    Japan|15197.340021014214|
|Australia|15402.999982118607|
+---------+------------------+



In [ ]:
### Question 2: Overall average number of affected users across attacks
avg_no_affected_users = attacks \
    .select(
        col("no_affected_users")
    ) \
    .agg({"no_affected_users": "avg"}) \
print("Question 2: Average Number of Affected Users")
avg_no_affected_users.show()



Question 2: Average Number of Affected Users
+---------------------+
|avg_no_affected_users|
+---------------------+
|    504684.1363333333|
+---------------------+



In [8]:
# OPTION B - SPARKSQL

stocks.createOrReplaceTempView("stocks")
query = spark.sql("""
    SELECT s.Name, s.date, s.volume
    FROM stocks s
    INNER JOIN (
        SELECT Name, MAX(volume) AS max_volume
        FROM stocks
        GROUP BY Name
    ) m
    ON s.Name = m.Name AND s.volume = m.max_volume
    ORDER BY s.volume DESC
""")

query.show()


+-----+----------+---------+
| Name|      date|   volume|
+-----+----------+---------+
|   VZ|2014-02-24|618237630|
|   GE|2015-11-17|431332632|
|  BAC|2016-02-11|375088650|
|   FB|2013-07-25|365380568|
|  PFE|2016-04-05|284468054|
|  MRO|2016-03-01|273996613|
|  AMD|2017-05-02|268336455|
| AAPL|2014-01-28|266833581|
|  KMI|2014-11-26|251563883|
| MSFT|2013-07-19|248354245|
| CSCO|2013-11-14|243255407|
|    F|2013-12-18|220362796|
| AMAT|2015-04-27|219415189|
|  DAL|2013-09-10|206363059|
|  CHK|2016-03-04|188966428|
| NFLX|2013-10-22|181099968|
|CMCSA|2014-02-13|166551204|
| ORCL|2013-07-12|157674694|
|   MU|2013-10-11|153906087|
|  FCX|2016-01-12|141597285|
+-----+----------+---------+
only showing top 20 rows



In [9]:
# Question 2
avg_volume_per_month = stocks \
    .select(
        date_format(col("date"), "MMM yyyy").alias("date"),  # I need to express dates as month values to group by those
        col("volume")
    ) \
    .groupBy(col("date")) \
    .agg(avg("volume").alias("avg_volume")) \
    .orderBy(col("date"), ascending=True)

In [10]:
avg_volume_per_month.show()

+--------+------------------+
|    date|        avg_volume|
+--------+------------------+
|Apr 2013| 4990292.249235766|
|Apr 2014| 4712436.740587831|
|Apr 2015|4049128.5425065733|
|Apr 2016| 4403273.369344325|
|Apr 2017|3879889.4020380294|
|Aug 2013|3936255.4692568844|
|Aug 2014|  3277133.94670969|
|Aug 2015| 4910543.650503291|
|Aug 2016|3645052.5863453816|
|Aug 2017|3479362.4257066296|
|Dec 2013|4092123.8206644976|
|Dec 2014|3991512.5363263786|
|Dec 2015| 4406736.627640037|
|Dec 2016|4130377.3358465354|
|Dec 2017|3919747.9897999605|
|Feb 2013|  5261788.80732293|
|Feb 2014| 4918514.594747739|
|Feb 2015| 4254824.743407599|
|Feb 2016| 5756484.877016129|
|Feb 2017| 4255274.505789474|
+--------+------------------+
only showing top 20 rows



In [11]:
# OPTION B - SPARKSQL

r2 = spark.sql("""

    SELECT 
        date_format(date, "MMM yyyy") AS month,
        AVG(volume) AS avg_volume
    FROM stocks
    GROUP BY date_format(date, "MMM yyyy")
    ORDER BY month ASC

""")

In [12]:
r2.show()

+--------+------------------+
|   month|        avg_volume|
+--------+------------------+
|Apr 2013| 4990292.249235766|
|Apr 2014| 4712436.740587831|
|Apr 2015|4049128.5425065733|
|Apr 2016| 4403273.369344325|
|Apr 2017|3879889.4020380294|
|Aug 2013|3936255.4692568844|
|Aug 2014|  3277133.94670969|
|Aug 2015| 4910543.650503291|
|Aug 2016|3645052.5863453816|
|Aug 2017|3479362.4257066296|
|Dec 2013|4092123.8206644976|
|Dec 2014|3991512.5363263786|
|Dec 2015| 4406736.627640037|
|Dec 2016|4130377.3358465354|
|Dec 2017|3919747.9897999605|
|Feb 2013|  5261788.80732293|
|Feb 2014| 4918514.594747739|
|Feb 2015| 4254824.743407599|
|Feb 2016| 5756484.877016129|
|Feb 2017| 4255274.505789474|
+--------+------------------+
only showing top 20 rows



In [13]:
# Question 3
avg_volume_per_tick = stocks \
    .select(
        col("Name"),
        col("volume")
    ) \
    .groupBy(col("Name")) \
    .agg(avg("volume").alias("avg_volume")) \
    .orderBy(col("avg_volume"), ascending=False)

In [14]:
avg_volume_per_tick.show()

+-----+--------------------+
| Name|          avg_volume|
+-----+--------------------+
|  BAC| 9.363379951628277E7|
| AAPL| 5.404789973550437E7|
|   GE| 4.211568283240667E7|
|    F|  3.44623748943606E7|
|   FB| 3.435926520969023E7|
| MSFT| 3.386946274583002E7|
|  AMD| 3.251903779666402E7|
|   MU| 3.024841118903892E7|
| INTC|2.9326713900714852E7|
| CSCO|2.8654349631453533E7|
|  PFE|2.8218910923749007E7|
|    T|2.5298330478951547E7|
|  CHK|2.4957711172359016E7|
|CMCSA|2.4329297557585385E7|
|  FCX| 2.354921657426529E7|
|    C| 2.068675837807784E7|
|  WFC|1.8511480962668784E7|
|  JPM|1.6589033246227164E7|
|   RF|1.6398451548848292E7|
| NFLX|1.6208007963463066E7|
+-----+--------------------+
only showing top 20 rows



In [15]:
# Question 4

# Step 0: Create a new DataFrame with the spread column
stocks_with_spread = stocks.withColumn("spread", col("high") - col("low"))

# Step 1: Compute max spread per stock
max_spread_per_stock = stocks_with_spread.groupBy("Name") \
    .agg(max("spread").alias("max_spread"))

# Step 2: Alias both DataFrames before join
stocks_alias = stocks_with_spread.alias("s")
max_spread_alias = max_spread_per_stock.alias("m")

# Step 3: Join and select desired columns
max_spread_dates = stocks_alias \
    .join(
        max_spread_alias,
        (col("s.Name") == col("m.Name")) &
        (col("s.spread") == col("m.max_spread")),
        how="inner"
    ) \
    .select(
        col("s.Name").alias("Name"),
        col("s.date").alias("Date"),
        col("s.spread").alias("Spread")
    ) \
    .orderBy(col("Spread").desc())

max_spread_dates.show()


+-----+----------+----------+
| Name|      Date|    Spread|
+-----+----------+----------+
| AMZN|2018-02-05| 138.26001|
| PCLN|2017-11-07|109.900024|
|  CMG|2015-11-20|      80.5|
|  AZO|2017-12-05| 59.695007|
| GOOG|2018-02-06| 58.569946|
|GOOGL|2018-02-05|     58.25|
| REGN|2015-09-28|    56.905|
| PRGO|2015-04-08| 51.097992|
|  HUM|2015-05-29| 43.384995|
| ULTA|2015-08-24|  43.22001|
|  MCK|2015-08-24|   42.1799|
|  HCA|2015-08-24|     42.06|
| BIIB|2014-03-21| 41.920013|
| ALGN|2017-12-04| 35.249985|
|  BLK|2018-02-05|  34.53003|
|  ADS|2016-01-28| 33.569992|
|  MTD|2018-02-05|  33.25952|
|   BA|2018-02-05| 32.609985|
|  AGN|2015-10-21| 32.499893|
|  TDG|2017-01-20|     32.25|
+-----+----------+----------+
only showing top 20 rows



In [93]:
# Question 5
avg_volume_per_tick = stocks \
    .select(
        col("Name"),
        col("close")
    ) \
    .filter(date_format(col("date"), "yyyy") == 2016) \
    .groupBy(col("Name")) \
    .agg(avg("close").alias("avg_stock_price")) \
    .orderBy(col("avg_stock_price"), ascending=False) 

avg_volume_per_tick.show()

+-----+------------------+
| Name|   avg_stock_price|
+-----+------------------+
| PCLN|1346.7126984126985|
|  AZO| 768.8588492063492|
|GOOGL| 763.2142063492068|
| GOOG| 743.4851190476194|
| AMZN| 699.5231349206346|
|  CMG|428.56464285714287|
| REGN|394.49996031746036|
|  MTD| 375.5649206349207|
|  BLK|350.08837301587295|
| EQIX|343.77674603174603|
| BIIB| 280.1989682539682|
|  SHW| 277.9422222222223|
| ORLY|269.24904761904753|
|  TDG|251.80250000000007|
|  AGN|241.69857142857143|
|  AYI|241.64035714285708|
|  PSA|238.75015873015872|
|  LMT|237.99261904761906|
| CHTR|229.49678571428566|
| ULTA|224.08111111111108|
+-----+------------------+
only showing top 20 rows



In [16]:
# Question 6

all_tickers = stocks \
    .select(col("Name")) \
    .distinct() 

all_tickers.show()

+----+
|Name|
+----+
|ALXN|
| GIS|
|   K|
| LEN|
|SPGI|
| AIV|
| AVY|
|BF.B|
| MMM|
| PKI|
| PPG|
|  RF|
| AXP|
|  CI|
| IRM|
| WEC|
|INFO|
| PFG|
|  PM|
| SNA|
+----+
only showing top 20 rows

